In [12]:
import itertools
import logging
import math
import os
from abc import abstractmethod
from random import random

import matplotlib.pyplot as plt
import numpy as np
import pyproj
import rasterio
from PIL import Image
import pandas as pd

In [13]:
#RasterPatchProvider
data_path = 'data/sample_data/' # root path of the data
raster_path = data_path+'EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio1.tif'
size=128
spatial_noise=0
normalize=True
fill_zero_if_error=False
nan_value=0
transformer = None
name = os.path.basename(os.path.splitext(raster_path)[0])

In [19]:
with rasterio.open(raster_path) as src:
    # read the metadata of the file
    meta = src.meta
    meta.update(count=src.count) # update the count of the meta to match the number of layers
    print('META:', meta)

    # read the data from the raster
    data = src.read()
    print('SHAPE:', data.shape)

    # get the NoData value from the raster
    nodata_value = src.nodatavals
    print('NODATAVALUE:', nodata_value)

    # iterate through all the layers
    for i in range(src.count):
        print(i)
        # replace the NoData values with np.nan
        data = data.astype(float)
        data[i] = np.where(data[i] == nodata_value[i], np.nan, data[i])
        if normalize:
            data[i] = (data[i] - np.nanmean(data[i]))/np.nanstd(data[i])
        data[i] = np.where(np.isnan(data[i]), nan_value, data[i])
            
    nb_layers = src.count
    print('NB LAYERS:', nb_layers)

    x_min = src.bounds.left
    y_min = src.bounds.bottom
    x_resolution = src.res[0]
    y_resolution = src.res[1]
    n_rows = src.height
    n_cols = src.width
    crs = src.crs
    print(x_min, y_min, x_resolution, y_resolution, n_rows, n_cols, crs)
    if nb_layers > 1:
        bands_names = [name+'_'+str(i+1) for i in range(nb_layers)]
    else:
        bands_names = [name]
        
    epsg = crs.to_epsg()
    if epsg != 4326:
        # create a pyproj transformer object to convert lat, lon to EPSG:32738
        transformer = pyproj.Transformer.from_crs("epsg:4326", epsg, always_xy=True)


META: {'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 8143, 'height': 5467, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, -32.266806146449994,
       0.0, -0.0083333333, 72.19152712905)}
SHAPE: (1, 5467, 8143)
NODATAVALUE: (None,)
0
NB LAYERS: 1
-32.266806146449994 26.63319397795 0.0083333333 0.0083333333 5467 8143 EPSG:4326


In [20]:
#get item

BoundingBox(left=-32.266806146449994, bottom=26.63319397795, right=35.591526915450004, top=72.19152712905)